<a href="https://colab.research.google.com/github/alanherprovat/ai-project/blob/main/Bi%20Lingual%20Fake%20News%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
import nltk
from nltk.corpus import stopwords
import re
import string

In [ ]:
#load dataset 
from google.colab import files
#u1= files.upload()
#u2= files.upload()
url1='https://raw.githubusercontent.com/alanherprovat/ai-project/main/eglishdata.csv'
url2='https://raw.githubusercontent.com/alanherprovat/ai-project/main/spanishdata.csv'
df1=pd.read_csv(url1)
df2=pd.read_csv(url2)


In [ ]:
#print first & last 5 rows of datasets
df1.head(5)
#df1.tail(5)#,df2.head(5),df2.tail(5)
#print last 5 rows of datasets


,title,body,label,link
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,http://www.bbc.com/news/world-us-canada-414191...
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,https://www.reuters.com/article/us-filmfestiva...
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,https://www.nytimes.com/2017/10/09/us/politics...
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,https://www.reuters.com/article/us-mexico-oil-...
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,http://www.cnn.com/videos/cnnmoney/2017/10/08/...


In [ ]:
#PRINT Shapes(no of rows and collmn)
df1.shape,df2.shape

((4009, 4), (3003, 6))

In [ ]:
#check for duplicate and drop them
df1.drop_duplicates(inplace= True)
df2.drop_duplicates(inplace= True)

#after duplicate checking
df1.shape,df2.shape

((3919, 4), (3002, 6))

In [ ]:
df1.tail(10),df2.tail(10)

(                                                  title  \
 3999    Red Sox fire manager Farrell after five seasons   
 4000  35 False Matches and 1 Erroneous Arrest As Pol...   
 4001  Hervé Leroux, Creator of the Hervé Léger Banda...   
 4002     The price tag for Pence's trip to Indianapolis   
 4003  CNN and Globalist Exposed - Steve Quayle and A...   
 4004                                    Trends to Watch   
 4005  Trump Jr. Is Soon To Give A 30-Minute Speech F...   
 4006        Ron Paul on Trump, Anarchism & the AltRight   
 4007  China to accept overseas trial data in bid to ...   
 4008  Vice President Mike Pence Leaves NFL Game Beca...   
 
                                                    body  label  \
 3999  FILE PHOTO: Oct 25, 2013; St. Louis, MO, USA; ...      1   
 4000  By Nicholas West\nAll things biometric are swe...      0   
 4001  Mr. Leroux was born Hervé Peugnet on May 30, 1...      1   
 4002  Ethical questions loom after Vice President Mi...      1   
 40

In [ ]:
#converting text  value of spanish label collumn to integer

for i in range(len(df2)):
  p=df2['label'].values
  if(p[i]=="True"):
    df2['label'].replace({"True": 1},inplace=True)
  elif(p[i]=="Fake"):
    df2['label'].replace({"Fake": 0},inplace=True)

df2.head(1000)

,title,body,label,link,Topic,Source
0,Goirigolzarri aplaza la fusión de BMN y Bankia...,BANCA Tras el mandato del Frob Goirigolzarri a...,1,http://www.elmundo.es/economia/2016/10/04/57f3...,Economy,Europa Press
1,"Evo Morales critica a Almagro por su ""conducta...","El presidente de Bolivia, Evo Morales, ha crit...",1,http://www.europapress.es/internacional/notici...,Politics,Europa Press
2,Errejón cree que PP y PSOE deben de estar nego...,Política Errejón cree que PP y PSOE deben de e...,1,http://www.eldiario.es/politica/Errejon-PP-PSO...,Politics,Europa Press
3,¿Adivinas cuántos países no tienen un Banco Ce...,¿Adivinas cuántos países no tienen un Banco Ce...,0,http://beforeitsnews.com/alternative/2015/06/a...,Economy,beforeitsnews.com
4,Marchamalo entra en las bibliotecas de escrito...,El periodista Jesús Marchamalo ha entrado de n...,1,http://www.europapress.es/cultura/libros-00132...,Entertainment,Europa Press
...,...,...,...,...,...,...
995,"Bruselas abre la puerta a ""explorar"" el coste ...",La Comisión Europea ha abierto la puerta este ...,1,http://www.europapress.es/economia/transportes...,Economy,Europa Press
996,Todas las solicitudes de nacionalidad por resi...,Todas las solicitudes de nacionalidad por resi...,1,http://www.europapress.es/sociedad/noticia-tod...,Society,Europa Press
997,"Héctor Alterio experimenta ""el tierno viaje de...",El Teatro Bellas Artes de Madrid ha presentado...,1,http://www.europapress.es/cultura/exposiciones...,Entertainment,Europa Press
998,El Real Madrid busca crecer ante un Athletic m...,El Real Madrid buscará este domingo en el esta...,1,http://www.europapress.es/deportes/futbol-0016...,Sport,Europa Press


In [ ]:
#### creating the manual testing csv file
from google.colab import files
df1_manual=df1.tail(10)
df2_manual=df2.tail(10)

manual_test=pd.concat([df1_manual,df2_manual],axis=0)
manual_test.to_csv('manual_testing.csv')
files.download('manual_testing.csv')

for i in range(4008,3998,-1):
  df1.drop([i],axis=0,inplace=True)
for i in range(3002,2992,-1):
  df2.drop([i],axis=0,inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#after dropping the tails data 
df1.tail(10),df2.tail(10)

(                                                  title  \
 3989  Margo Price, Nashville Outsider, Tells It Like...   
 3990  Saturday: Two To Go - Jamo v Max Scherzer, Lin...   
 3991                          Matchup: Bulls v Mud Hens   
 3992  Iraq War Vet Gave Exact Location of Vegas Shoo...   
 3993  10/2 Happenings: HBD Ernie,Honest Eddie, Rex, ...   
 3994  Smart Meters Create A “Honeypot” Of Personal I...   
 3995  Harvey Weinstein scandal: What next for Hollyw...   
 3996                                 The Spy in the IRA   
 3997  Trump Takes Bold Action to Save U.S. From Euro...   
 3998  9/28 Through the 40s: The Gloaming; HBD Bill, ...   
 
                                                    body  label  \
 3989  “I think all that strife, it kind of helped me...      1   
 3990  Saturday: Two To Go – Jamo v Max Scherzer, Lin...      0   
 3991  Matchup: Bulls v Mud Hens\n% of readers think ...      0   
 3992  By Jack Burns\n \nThe response time of the Las...      0   
 39

In [ ]:
df1.shape,df2.shape

((3909, 4), (2992, 6))

In [ ]:
merged_data = pd.concat([df1,df2])
#shape of merged csv files
merged_data.shape

(6901, 6)

In [ ]:
#**merged inforamtion**
merged_data.head(6000)

,title,body,label,link,Topic,Source
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,http://www.bbc.com/news/world-us-canada-414191...,NaN,NaN
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,https://www.reuters.com/article/us-filmfestiva...,NaN,NaN
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,https://www.nytimes.com/2017/10/09/us/politics...,NaN,NaN
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,https://www.reuters.com/article/us-mexico-oil-...,NaN,NaN
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,NaN,NaN
...,...,...,...,...,...,...
2087,Draghi dice que el BCE no ha discutido el fina...,El Consejo de Gobierno del Banco Central Europ...,1,http://www.europapress.es/economia/finanzas-00...,Economy,Europa Press
2088,"PP pide a PNV que sea ""serio"" y no ""maree a la...","Política PP pide a PNV que sea ""serio"" y no ""m...",1,http://www.eldiario.es/politica/PP-PNV-PSE-EH-...,Politics,Europa Press
2089,"Viajes: En Ecuador, joyas escondidas en la cos...",Mi cabaña en Playa Escondida era rústica pero ...,1,https://www.nytimes.com/2015/03/08/universal/e...,Society,nytimes.com
2090,Nicolás Maduro también asume control de selecc...,Ya no es solo el poder judicial y la Asamblea....,0,http://www.actualidadpanamericana.com/nicolas-...,Sport,actualidadpanamericana.com


In [ ]:
ds = merged_data.sample(frac=1)
ds.head(20)
#ds.head(200)


,title,body,label,link,Topic,Source
2617,"U.S. Helicopter Crashes on Okinawa, Adding to ...",TOKYO — A transport helicopter made an emergen...,1,https://www.nytimes.com/2017/10/11/world/asia/...,NaN,NaN
1585,"Euskadi, entre las 7 comunidades que crecerán ...","Euskadi, Baleares, Castilla-La Mancha, Cataluñ...",1,http://www.europapress.es/euskadi/noticia-eusk...,Economy,Europa Press
2608,Navarro rescata al Barcelona ante el Unics Kazan,Navarro rescata al Barcelona ante el Unics Kaz...,1,http://www.europapress.es/deportes/baloncesto-...,Sport,Europa Press
1111,Refugees - and forests - benefit as Kenya's he...,"KAKUMA, Kenya (Thomson Reuters Foundation) - T...",1,https://www.reuters.com/article/us-kenya-energ...,NaN,NaN
1599,‘Dynasty’ or ‘Dallas’? Squaring Off in the Lil...,JACOBS A fig stuffed with goat cheese and wrap...,1,https://www.nytimes.com/2017/10/12/arts/televi...,NaN,NaN
1103,Desmienten que Santos haya usado el papamóvil ...,Punto final a los rumores sobre un supuesto us...,0,http://www.actualidadpanamericana.com/desmient...,Politics,actualidadpanamericana.com
1785,Devuelven transacción de Isagen porque gobiern...,Amargo sinsabor deja en el gobierno nacional u...,0,http://www.actualidadpanamericana.com/devuelve...,Politics,actualidadpanamericana.com
1536,How Mel Brooks' lady of dance made it in a man...,Image copyright Target Live Image caption Mel ...,1,http://www.bbc.com/news/entertainment-arts-415...,NaN,NaN
2959,El Real Madrid busca puntos obligados y con bu...,El Real Madrid buscará este martes en el estad...,1,http://www.europapress.es/deportes/futbol-0016...,Sport,Europa Press
2732,"Casado, sobre el 'powerpoint' para instruir en...","Política Casado, sobre el 'powerpoint' para in...",1,http://www.eldiario.es/politica/Casado-powerpo...,Politics,Europa Press


**dropping the title,link,topic,source collumns**


In [ ]:
data=ds.drop(['title','link','Topic','Source'],axis=1)
data.head(10)

,body,label
2617,TOKYO — A transport helicopter made an emergen...,1
1585,"Euskadi, Baleares, Castilla-La Mancha, Cataluñ...",1
2608,Navarro rescata al Barcelona ante el Unics Kaz...,1
1111,"KAKUMA, Kenya (Thomson Reuters Foundation) - T...",1
1599,JACOBS A fig stuffed with goat cheese and wrap...,1
1103,Punto final a los rumores sobre un supuesto us...,0
1785,Amargo sinsabor deja en el gobierno nacional u...,0
1536,Image copyright Target Live Image caption Mel ...,1
2959,El Real Madrid buscará este martes en el estad...,1
2732,"Política Casado, sobre el 'powerpoint' para in...",1


In [ ]:
#cheking for null contents
data.isnull().sum(),data.shape

(body     26
 label     0
 dtype: int64, (6901, 2))

In [ ]:
#removing the rows with null contents
data.dropna(subset=['body'], inplace=True)
data.isnull().sum(),data.shape

(body     0
 label    0
 dtype: int64, (6875, 2))

In [ ]:

nltk.download('stopwords')
from nltk.stem import PorterStemmer
port_stem=PorterStemmer()
#Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers.
# A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate”
#stopwords doesnot add much meaning to a sentence (is,a, the etc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def process(body):
  #remove puntuation
  nopun=re.sub('[^a-zA-Zá-ü]',' ',body)
  nopun=nopun.lower()
  nopun=nopun.split()
  nopun=[port_stem.stem(word) for word in nopun if word  not in stopwords.words(['english','spanish'])]
  nopun=' '.join(nopun)

  return nopun

In [ ]:
#removing all kinds of un necessary characters and stop words
data['body']=data['body'].apply(process)



In [ ]:
print(data['body'])

2617    tokyo transport helicopt made emerg land catch...
1585    euskadi balear castilla mancha cataluña comuni...
2608    navarro rescata barcelona unic kazan publicado...
1111    kakuma kenya thomson reuter foundat turkana co...
1599    jacob fig stuf goat chees wrap prosciutto play...
                              ...                        
3940    cnn massacr vega nation tragedi famili one ste...
1578    get done black hole news offens throw outsid n...
2387    obama deep state appointe burrow govern news g...
3855    isaac davi cannabi legal gain ground unit stat...
1551    tra secuencia captada partido equipo empató do...
Name: body, Length: 6875, dtype: object


In [ ]:
x=data['body'].values
y=data['label'].values
print(x),print(y),x.shape,y.shape

['tokyo transport helicopt made emerg land catch fire wednesday unit state militari train area northern part okinawa japan stir concern deploy craft near civilian area statement third marin expeditionari forc said helicopt take part routin train none crew member civilian injur accord japanes defens ministri helicopt ch made land taka district higashi villag japanes govern build helicopt land pad american militari project small fierc group resid protest year nhk public broadcast quot unit state militari say helicopt station futenma air base okinawa advertis continu read main stori itsunori onodera japan defens minist told report govern investig whether civilian hurt mr onodera said japan ask unit state immedi investig caus prevent recurr thoroughli implement safeti control suppli detail inform'
 'euskadi balear castilla mancha cataluña comunitad valenciana galicia madrid siet comunidad autónoma crecerán encima media año seguida region aragón navarra mayoría superará media próximo año se

(None, None, (6875,), (6875,))

In [ ]:

#converting the full body text to a matrix of tokens
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x=vectorizer.fit_transform(x)
#vectorizer.get_feature_names_out()
#vectorizer.vocabulary_
print(x)

  (0, 38118)	0.05042427582208952
  (0, 21952)	0.059532615043427335
  (0, 69646)	0.07092703487874205
  (0, 17109)	0.04857429677081841
  (0, 64152)	0.0653791202391223
  (0, 37098)	0.07437644147946958
  (0, 71418)	0.09738435635123042
  (0, 60494)	0.10016121655081357
  (0, 57193)	0.0653791202391223
  (0, 12577)	0.05635494820270157
  (0, 36965)	0.061749743839117334
  (0, 6072)	0.05032472521522529
  (0, 48639)	0.05677405364825163
  (0, 36337)	0.068745164009828
  (0, 77036)	0.053655037052886406
  (0, 39400)	0.1127851064436381
  (0, 61763)	0.04013910871927332
  (0, 71895)	0.044664915184492275
  (0, 47456)	0.06158734738113671
  (0, 51713)	0.24561561478417876
  (0, 39850)	0.12280780739208938
  (0, 68719)	0.03858808647318072
  (0, 44840)	0.052372202302952635
  (0, 59769)	0.04673832918516042
  (0, 16915)	0.04342548888467841
  :	:
  (6874, 37381)	0.08293840210003882
  (6874, 39267)	0.0569627281927473
  (6874, 22255)	0.07334648149086237
  (6874, 59099)	0.04842486511709925
  (6874, 69251)	0.052149296

In [ ]:
#train and test data
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.25, random_state=0)

**NAive bayes classifier model**


In [ ]:
# naive bayes classifier model
from sklearn.naive_bayes import MultinomialNB
cs=MultinomialNB()
classifier= MultinomialNB().fit(x_train,y_train)



In [ ]:
##classification report of naive bayes model
from sklearn.metrics import classification_report,accuracy_score
npred=classifier.predict(x_test)
print(classification_report(y_test,npred))
#print(npred)
print('Accuracy of naive based Model:: ',accuracy_score(y_test,npred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96       912
           1       0.93      0.97      0.95       807

    accuracy                           0.95      1719
   macro avg       0.95      0.96      0.95      1719
weighted avg       0.95      0.95      0.95      1719

Accuracy of naive based Model::  0.954043048283886


**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)
#lr.score(x_test,y_test)



LogisticRegression()

In [ ]:
lr_pred=lr.predict(x_test)
print(classification_report(y_test,lr_pred))

print("accuracy of logistic regression model:: ",accuracy_score(y_test,lr_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       912
           1       0.98      0.98      0.98       807

    accuracy                           0.98      1719
   macro avg       0.98      0.98      0.98      1719
weighted avg       0.98      0.98      0.98      1719

accuracy of logistic regression model::  0.9802210587550901


**decision tree classification**



In [ ]:

from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
#dt.score(x_test,y_test)


DecisionTreeClassifier()

In [ ]:


dt_pred=dt.predict(x_test)
print(classification_report(y_test,dt_pred))

print("accuracy of decision tree model:: ",accuracy_score(y_test,dt_pred))



              precision    recall  f1-score   support

           0       0.97      0.97      0.97       912
           1       0.97      0.97      0.97       807

    accuracy                           0.97      1719
   macro avg       0.97      0.97      0.97      1719
weighted avg       0.97      0.97      0.97      1719

accuracy of decision tree model::  0.9685863874345549


**Gradient Boosting Classifier**

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
gc=GradientBoostingClassifier(random_state=0)
gc.fit(x_train,y_train)
#gc.score(x_test,y_test)


GradientBoostingClassifier(random_state=0)

In [ ]:

gc_pred=gc.predict(x_test)
print(classification_report(y_test,gc_pred))

print("accuracy of gradient boosting  model:: ",accuracy_score(y_test,gc_pred))


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       912
           1       0.97      0.98      0.98       807

    accuracy                           0.98      1719
   macro avg       0.98      0.98      0.98      1719
weighted avg       0.98      0.98      0.98      1719

accuracy of gradient boosting  model::  0.9778941244909831


**RAndom forest classifier**


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
#rfc.score(x_test,y_test)

RandomForestClassifier()

In [ ]:
rfc_pred=gc.predict(x_test)
print(classification_report(y_test,rfc_pred))

print("accuracy of Random Forest  model:: ",accuracy_score(y_test,gc_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       912
           1       0.97      0.98      0.98       807

    accuracy                           0.98      1719
   macro avg       0.98      0.98      0.98      1719
weighted avg       0.98      0.98      0.98      1719

accuracy of Random Forest  model::  0.9778941244909831


**Manual Testing Section**

In [ ]:
true = [0]
false = [0]
def output_lable(n):
    if n == 0:
      false[0]+=1
      return "Fake News"
    elif n == 1:
      true[0]+=1
      return "It's a Valid News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(process) 
    new_x_test = new_def_test["text"]
    new_xv_test =vectorizer.transform(new_x_test)
    pred_nbc= classifier.predict(new_xv_test)
    pred_LR = lr.predict(new_xv_test)
    pred_DT = dt.predict(new_xv_test)
    pred_GBC = gc.predict(new_xv_test)
    pred_RFC = rfc.predict(new_xv_test)
    return print("\n\nNB Prediction: {} \nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_nbc[0]),output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = (input())
news=str(news)
manual_testing(news)

In [ ]:
print(true)
print(false)
if(true[0]>false[0]):
  print("Approximately It's a  Valid News")
else:
  print("Approximately It's a  Fake News")
  